In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [2]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [3]:
print(sentence)
print(len(sentence))

Brick walls are there for a reason and you must not think that the brick walls aren't there to keep us out, but rather in this way that the brick walls are there to show us how badly we want things.
198


In [4]:
#2. 문자 집합 만들기
world_set = list(set(sentence))
print(world_set)
## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
# vocab = {char: idx for idx, char in enumerate(sorted(world_set))} # 코드? 똑같은 출력이 안나옴
vocab = {char: idx for idx, char in enumerate(world_set)}
# sorted 쓰면 안될듯.. 마지막 문장 출력에서 큰 변화

['B', 'n', 'b', 'w', '.', 'r', 'u', 'e', 'o', "'", 'c', 'h', 'i', 'f', 'd', 'k', ' ', 'p', 'g', 'a', 't', 'l', 's', ',', 'y', 'm']


In [5]:
print(vocab)

# 올바른 출력: {',': 0, 'r': 1, 'i': 2, 's': 3, "'": 4, 'm': 5, 'l': 6, 'a': 7, 'p': 8, 'o': 9, 't': 10, 'h': 11, ' ': 12, 'e': 13, 'y': 14, 'd': 15, 'B': 16, 'n': 17, 'b': 18, '.': 19, 'w': 20, 'k': 21, 'c': 22, 'g': 23, 'f': 24, 'u': 25}

{'B': 0, 'n': 1, 'b': 2, 'w': 3, '.': 4, 'r': 5, 'u': 6, 'e': 7, 'o': 8, "'": 9, 'c': 10, 'h': 11, 'i': 12, 'f': 13, 'd': 14, 'k': 15, ' ': 16, 'p': 17, 'g': 18, 'a': 19, 't': 20, 'l': 21, 's': 22, ',': 23, 'y': 24, 'm': 25}


In [6]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [7]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [8]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i : i+sequence_length]
  y_str = sentence[i+1 : i+sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [9]:
x_data

# Brick wall
# rick walls
# ick walls ....

[[0, 5, 12, 10, 15, 16, 3, 19, 21, 21],
 [5, 12, 10, 15, 16, 3, 19, 21, 21, 22],
 [12, 10, 15, 16, 3, 19, 21, 21, 22, 16],
 [10, 15, 16, 3, 19, 21, 21, 22, 16, 19],
 [15, 16, 3, 19, 21, 21, 22, 16, 19, 5],
 [16, 3, 19, 21, 21, 22, 16, 19, 5, 7],
 [3, 19, 21, 21, 22, 16, 19, 5, 7, 16],
 [19, 21, 21, 22, 16, 19, 5, 7, 16, 20],
 [21, 21, 22, 16, 19, 5, 7, 16, 20, 11],
 [21, 22, 16, 19, 5, 7, 16, 20, 11, 7],
 [22, 16, 19, 5, 7, 16, 20, 11, 7, 5],
 [16, 19, 5, 7, 16, 20, 11, 7, 5, 7],
 [19, 5, 7, 16, 20, 11, 7, 5, 7, 16],
 [5, 7, 16, 20, 11, 7, 5, 7, 16, 13],
 [7, 16, 20, 11, 7, 5, 7, 16, 13, 8],
 [16, 20, 11, 7, 5, 7, 16, 13, 8, 5],
 [20, 11, 7, 5, 7, 16, 13, 8, 5, 16],
 [11, 7, 5, 7, 16, 13, 8, 5, 16, 19],
 [7, 5, 7, 16, 13, 8, 5, 16, 19, 16],
 [5, 7, 16, 13, 8, 5, 16, 19, 16, 5],
 [7, 16, 13, 8, 5, 16, 19, 16, 5, 7],
 [16, 13, 8, 5, 16, 19, 16, 5, 7, 19],
 [13, 8, 5, 16, 19, 16, 5, 7, 19, 22],
 [8, 5, 16, 19, 16, 5, 7, 19, 22, 8],
 [5, 16, 19, 16, 5, 7, 19, 22, 8, 1],
 [16, 19, 16, 5, 7,

In [10]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

'''
올바른 출력
[16, 1, 2, 22, 21, 12, 20, 7, 6, 6]
[1, 2, 22, 21, 12, 20, 7, 6, 6, 3]
'''

[0, 5, 12, 10, 15, 16, 3, 19, 21, 21]
[5, 12, 10, 15, 16, 3, 19, 21, 21, 22]


'\n올바른 출력\n[16, 1, 2, 22, 21, 12, 20, 7, 6, 6]\n[1, 2, 22, 21, 12, 20, 7, 6, 6, 3]\n'

In [11]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = np.eye(len(vocab))[x_data]

# vocab 딕셔너리의 길이를 기반으로 identity 행렬을 생성 (행렬의 크기: vocab의 길이 * vocab의 길이)
# identity 행렬에 x_data를 인덱싱하여 각 샘플에 대해 원핫인코딩 수행


print(x_one_hot[0], len(x_one_hot[0]))
# [4, 20, 13, 7, 14, 0, 24, 5, 15, 15] 을 identity 행렬로 반환

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot) # 입력: 실수형
Y = torch.LongTensor(y_data) # 레이블: 정수형

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0.]] 10


In [12]:
print(len(X), len(X[0]))

188 10


In [13]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

'''
훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])
'''

훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


'\n훈련 데이터의 크기 : torch.Size([188, 10, 26])\n레이블의 크기 : torch.Size([188, 10])\n'

In [14]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [15]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([ 5, 12, 10, 15, 16,  3, 19, 21, 21, 22])


In [16]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)


class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers, vocab_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.fc(x)
        return x



In [17]:
net = Net(vocab_size, hidden_size, 2, vocab_size)

In [18]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [19]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

# print(outputs.reshape(X.size(0), -1, input_dim).shape)


torch.Size([188, 10, 26])


In [20]:
##15. Training 시작

for i in range(200):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

ooBoBooooooooo.ooo.o.ooooosoo.soooooosooooosoBoosooooosoooooooooo.ooo.oBooooooooo.sooooo.o.ooooo.soooBoooosooooooooo.oosoooosooooooooooooo.ooo.oBooooooooo.ooo.o.oooosooooooooooooo.oooo.oooooooosoos
o                                                                                                                                                                                                    
                                                                                                                                                                                                     
                                                                                                                                                                                                     
                                                                                                                                                                                                     
          

In [21]:
predict_str

# 에폭 200번
# Brick walls are there for a reason and you must not think that the brick walls aren't there to keep us out, but rather in this way that the brick walls are there to show us how badly we want things.


"rick walls are there tor a reason and you must not think that the brick walls are 't there to keep us out, but rather in this way that the brick walls are there to khow us how badly we want things."

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요!

영어가 아닌 한국어로 시도해보는 것도 좋겠죠?

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

### 영어문장



- sequence length가 너무 길거나 짧을 경우
overfit, underfit 가능성 존재

In [3]:

#1. 생성할 문장 데이터

sentence = ("Killing me slow, out the window "
            "I'm always waiting for you to be waiting below "
            "Devils roll the dice, angels roll their eyes "
            "What doesn't kill me makes me want you more")

#2. 문자 집합 만들기
world_set = list(set(sentence))
print(world_set)
vocab = {char: idx for idx, char in enumerate(world_set)}

#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

#4. 하이퍼 파라미터 설정

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i : i+sequence_length]
  y_str = sentence[i+1 : i+sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = np.eye(len(vocab))[x_data]

# vocab 딕셔너리의 길이를 기반으로 identity 행렬을 생성 (행렬의 크기: vocab의 길이 * vocab의 길이)
# identity 행렬에 x_data를 인덱싱하여 각 샘플에 대해 원핫인코딩 수행


print(x_one_hot[0], len(x_one_hot[0]))
# [4, 20, 13, 7, 14, 0, 24, 5, 15, 15] 을 identity 행렬로 반환

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot) # 입력: 실수형
Y = torch.LongTensor(y_data) # 레이블: 정수형


##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))




##9.원핫인코딩 결과 샘플 확인하기
print(X[0])


##10. 레이블 데이터 샘플 확인하기
print(Y[0])

##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers, vocab_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.fc(x)
        return x


net = Net(vocab_size, hidden_size, 2, vocab_size)

##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

['i', 'K', 'a', 'k', ',', 't', ' ', 'I', 'l', 'd', 'b', 'h', 'y', 'v', 'D', 'c', 'r', 'e', 'g', 'm', 's', "'", 'u', 'o', 'W', 'n', 'w', 'f']
문자 집합 크기 : 28
0 Killing me -> illing me 
1 illing me  -> lling me s
2 lling me s -> ling me sl
3 ling me sl -> ing me slo
4 ing me slo -> ng me slow
5 ng me slow -> g me slow,
6 g me slow, ->  me slow, 
7  me slow,  -> me slow, o
8 me slow, o -> e slow, ou
9 e slow, ou ->  slow, out
10  slow, out -> slow, out 
11 slow, out  -> low, out t
12 low, out t -> ow, out th
13 ow, out th -> w, out the
14 w, out the -> , out the 
15 , out the  ->  out the w
16  out the w -> out the wi
17 out the wi -> ut the win
18 ut the win -> t the wind
19 t the wind ->  the windo
20  the windo -> the window
21 the window -> he window 
22 he window  -> e window I
23 e window I ->  window I'
24  window I' -> window I'm
25 window I'm -> indow I'm 
26 indow I'm  -> ndow I'm a
27 ndow I'm a -> dow I'm al
28 dow I'm al -> ow I'm alw
29 ow I'm alw -> w I'm alwa
30 w I'm alwa -

In [10]:
#15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()


#16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

illing me slow, out the window I'm always waiting bor you to be waiting below Devils roll the dice, angels roll the r eyes What doesn't kill me makes me want you moro
illing me slow, out the window I'm always waiting bor you to be waiting below Devils roll the dice, angels roll the r eyes What doesn't kill me makes me want you moro
illing me slow, out the window I'm always waiting bor you to be waiting below Devils roll the dice, angels roll the r eyes What doesn't kill me makes me want you moro
illing me slow, out the window I'm always waiting bor you to be waiting below Devils roll the dice, angels roll the r eyes What doesn't kill me makes me want you moro
illing me slow, out the window I'm always waiting bor you to be waiting below Devils roll the dice, angels roll the r eyes What doesn't kill me makes me want you moro
illing me slow, out the window I'm always waiting bor you to be waiting below Devils roll the dice, angels roll the r eyes What doesn't kill me makes me want you mor

In [3]:


#1. 생성할 문장 데이터

sentence = ("새벽 그림자 사이에 "
            "뜀박질 하는 불빛은 "
            "모닥불 같아 "
            "선선한 아침의 노을 "
            "저기 아래는 우리의 보금자리야")

#2. 문자 집합 만들기
world_set = list(set(sentence))
print(world_set)
## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {char: idx for idx, char in enumerate(world_set)}

#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

#4. 하이퍼 파라미터 설정

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i : i+sequence_length]
  y_str = sentence[i+1 : i+sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = np.eye(len(vocab))[x_data]

# vocab 딕셔너리의 길이를 기반으로 identity 행렬을 생성 (행렬의 크기: vocab의 길이 * vocab의 길이)
# identity 행렬에 x_data를 인덱싱하여 각 샘플에 대해 원핫인코딩 수행


# print(x_one_hot[0], len(x_one_hot[0]))
# [4, 20, 13, 7, 14, 0, 24, 5, 15, 15] 을 identity 행렬로 반환

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot) # 입력: 실수형
Y = torch.LongTensor(y_data) # 레이블: 정수형


##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))




##9.원핫인코딩 결과 샘플 확인하기
# print(X[0])


##10. 레이블 데이터 샘플 확인하기
# print(Y[0])

##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers, vocab_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.fc(x)
        return x


net = Net(vocab_size, hidden_size, 2, vocab_size)

##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()


#16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

['노', '그', '아', '을', '는', '이', '닥', '래', '침', '의', '한', '질', '불', '벽', '금', '보', '에', '저', '새', '우', '리', '선', '야', '림', '하', ' ', '박', '자', '사', '기', '빛', '뜀', '은', '같', '모']
문자 집합 크기 : 35
0 새벽 그림자 사이에 -> 벽 그림자 사이에 
1 벽 그림자 사이에  ->  그림자 사이에 뜀
2  그림자 사이에 뜀 -> 그림자 사이에 뜀박
3 그림자 사이에 뜀박 -> 림자 사이에 뜀박질
4 림자 사이에 뜀박질 -> 자 사이에 뜀박질 
5 자 사이에 뜀박질  ->  사이에 뜀박질 하
6  사이에 뜀박질 하 -> 사이에 뜀박질 하는
7 사이에 뜀박질 하는 -> 이에 뜀박질 하는 
8 이에 뜀박질 하는  -> 에 뜀박질 하는 불
9 에 뜀박질 하는 불 ->  뜀박질 하는 불빛
10  뜀박질 하는 불빛 -> 뜀박질 하는 불빛은
11 뜀박질 하는 불빛은 -> 박질 하는 불빛은 
12 박질 하는 불빛은  -> 질 하는 불빛은 모
13 질 하는 불빛은 모 ->  하는 불빛은 모닥
14  하는 불빛은 모닥 -> 하는 불빛은 모닥불
15 하는 불빛은 모닥불 -> 는 불빛은 모닥불 
16 는 불빛은 모닥불  ->  불빛은 모닥불 같
17  불빛은 모닥불 같 -> 불빛은 모닥불 같아
18 불빛은 모닥불 같아 -> 빛은 모닥불 같아 
19 빛은 모닥불 같아  -> 은 모닥불 같아 선
20 은 모닥불 같아 선 ->  모닥불 같아 선선
21  모닥불 같아 선선 -> 모닥불 같아 선선한
22 모닥불 같아 선선한 -> 닥불 같아 선선한 
23 닥불 같아 선선한  -> 불 같아 선선한 아
24 불 같아 선선한 아 ->  같아 선선한 아침
25  같아 선선한 아침 -> 같아 선선한 아침의
26 같아 선선한 아침의 -> 아 선선한 아침의 
27 아 선선한 아침의  ->  선선한 아침의 노
28  선선한 아침의 노 -> 선선한 아침의 노을
29 선선한 아